# Valorant Victory Prediction

## Data Pre-processing

In [1]:
import numpy as np
import pandas as pd


### Pre-process player stats

In [2]:
acs = pd.read_csv('./Data files/acs.csv')
agent_wr = pd.read_csv('./Data files/agent-win-rates.csv')
agent_picks = pd.read_csv('./Data files/agent-picks.csv')
aim = pd.read_csv('./Data files/aim.csv')
assists = pd.read_csv('./Data files/assists.csv')
clutches = pd.read_csv('./Data files/clutches.csv')
damage = pd.read_csv('./Data files/damage.csv')
defusals = pd.read_csv('./Data files/defusals.csv')
headshots = pd.read_csv('./Data files/headshots.csv')
kd = pd.read_csv('./Data files/kd.csv')
multikill = pd.read_csv('./Data files/multi-kills.csv')
plants = pd.read_csv('./Data files/plants.csv')
spend = pd.read_csv('./Data files/spend.csv')
weapon_picks = pd.read_csv('./Data files/weapon-picks.csv')


### Pre-process team stats

In [3]:
team_attack_speed = pd.read_csv('./Data files/team-attack-speed.csv')
team_comp = pd.read_csv('./Data files/team-comp.csv')
team_map_performance = pd.read_csv('./Data files/team-map-performance.csv')
team_post_plants = pd.read_csv('./Data files/team-post-plants.csv')
team_round_performance = pd.read_csv('./Data files/team-round-performance.csv')
team_win_conditions = pd.read_csv('./Data files/team-win-conditions.csv')
team_xvy_performance = pd.read_csv('./Data files/team-xvy-performance.csv')